<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized%5D1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [ ]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
!pip install swmm5
!pip install openpyxl
import math
import itertools
import os, sys
import swmm5
from pyswmm import Simulation, Links
from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
from google.colab import files
import pandas as pd
import re
import copy

# File Upload

In [ ]:
uploaded = files.upload()

inp = SwmmInput("Inlet_Drains_Model1.inp")
out = SwmmOutput("Inlet_Drains_Model1.out")
rpt = SwmmReport("Inlet_Drains_Model1.rpt")
available_sizes = uploaded["Available Sizes.txt"]
print("Uploaded files:", os.listdir())
conduits_df = inp["CONDUITS"]
print(conduits_df)

# Boundary Parameters

In [73]:
peak_flow_dict = {}
peak_flow =  {}
peak_vel = {}
adjust_pipe_size = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
vel_min = 0.8
vel_max = 5
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}
trials = {}
velocity = {}

# Data Frame

In [74]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']
junctions_data = inp['JUNCTIONS']
outfalls_data = inp['OUTFALLS']

# Store results
conduit_info = []

# Accessing Data Dictionary

In [ ]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
  conduit_dict = vars(conduit_data) # Access all data in Conduit
  print (conduit_dict)

print(xsections)  # Print the content of the xsections InpSection object
for xsection_name, xsection_data in xsections.items():
  print(xsection_data)  # Print the name of each parameter in the xsections section

# Data Gathering

In [ ]:
with Simulation("Inlet_Drains_Model1.inp") as sim:
  for step in sim:
    pass

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  peak_flow = conduit_data.flow_max
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

  print (f"Section Name: {xsection_name}")
  print (f"Length: {length} m")
  print (f"Max Flow: {peak_flow} m3/s")
  print (f"Shape: {shape}")
  print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
  print (f"Inlet Offset: {in_offset} m")
  print (f"Outlet Offset: {out_offset} m")

# Optimization


Arrange Conduits based on Pipe Flow

In [ ]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None):
    """Traces the flow path upstream from a given node, including junctions."""
    if path is None:
        path = []

    path.append(start_node)  # Add the current node (junction or outfall) to the path

    # Find conduits connected to the current node as the downstream
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # If no upstream conduits are found, it's the outermost junction
    if not connected_conduits:
        return path  # Return the path including the outermost junction

    # Trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        new_path = trace_upstream_flow(upstream_node, path + [conduit_name])  # Include conduit in the path
        if new_path:
            return new_path

    return None  # Return None if no path is found (should not happen in a well-defined network)


flow_paths = {}
for outfall_node_name in outfall_node_names:
    flow_path = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = flow_path

# Show results
for outfall_node, flow_path in flow_paths.items():
    flow_path.reverse() # Reverse the flow path list
    print(f"Flow Path for {outfall_node}: {flow_path}")

    # Separate conduits and junctions for clearer output (optional)
    conduits = [item for item in flow_path if item in conduit_connections]
    junctions = [item for item in flow_path if item not in conduit_connections]
    print(f"Sorted Conduits for {outfall_node}: {conduits}")
    print(f"Junctions in Path: {junctions}")

Identify the Invert Elevations of the Conduits

In [ ]:
conduit_info = []
for conduit_name, conduit_data in inp['CONDUITS'].items():
    upstream_junction = conduit_data.from_node
    downstream_junction = conduit_data.to_node

    upstream_invert = junctions_data[upstream_junction].elevation

    # Check if downstream node is an outfall
    if downstream_junction in outfalls_data:
        downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
    elif downstream_junction in junctions_data:
        downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
    else:
        print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
        downstream_invert = None  # or some default value

    length = conduit_data.length

    if downstream_invert is not None:
        slope = (upstream_invert - downstream_invert) / length
    else:
        slope = None
    # Input all data gathered and solve in a DataFrame
    conduit_info.append([conduit_name, upstream_junction, upstream_invert, downstream_junction, downstream_invert, length, slope])

    print(f"Conduit: {conduit_name}; From: {upstream_junction} To: {downstream_junction}")
    print(f"Upstream Invert Elevation: {upstream_invert} m")
    print(f"Downstream Invert Elevation: {downstream_invert} m")
    print(f"Slope: {slope}")

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Upstream Invert', 'To Node', 'Downstream Invert', 'Length', 'Slope'])

conduit_df.to_excel("conduit_analysis_data.xlsx", index=False)



Continuity Equation

In [ ]:
# Function for adjusting Pipe Sizes
def adjust_pipe_size (peak_flow, peak_vel):
    Area = peak_flow / peak_vel
    new_diameter = math.sqrt( Area / (0.25 * math.pi))

    pass

Function: Identifying Peak Flow Rates

In [ ]:
def peak_flow(inp):
  with Simulation(inp) as sim:
    for step in sim:
      pass

    peak_flow = {}
    for conduit_name, conduit_data in inp["CONDUITS"]:
      peak_flow = conduit_data.flow_max
  return peak_flow

Function: Identifying Suitable Pipe Sizes

In [ ]:
def find_suitable_sizes(conduit_name, available_sizes, peak_flow):
  suitable_sizes = []
  with open(available_sizes) as file:
    for line in file:
      size = float(line.strip())
      velocity = peak_flow / (math.pi * size^2 * 0.25)
      if vel_min <= velocity <= vel_max:
        suitable_sizes.append(size)
  return suitable_sizes

Function: Construction Cost

In [ ]:
# Function for the Computation of Construction Costs
def construction_costs (unit_cost, length, size):
  pass

Function: Identifying the Largest Pipe Size and Minimum Slope

In [ ]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size(available_sizes):
  largest_size = 0.0
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

available_sizes = "Available Sizes.txt"
largest_size = largest_size(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

Initializing Pipe Sizes and Slopes

In [ ]:
# Set all pipe sizes to largest_size
for xsection_name, xsection_data in inp["CONDUITS"].items():
  xsection_data.height = largest_size(available_sizes)

# Saving data before Simulation
inp.write_file("Modified_Inlet_Drains_Model1.inp")

Simulation of Data before Optimization

In [ ]:
# Simulation of Data
with Simulation("Modified_Inlet_Drains_Model1.inp") as sim:
  for step in sim:
    pass

Optimization of Pipe Sizes and Slopes

In [ ]:
# Optimization of Sizes and Slopes
for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  peak_flow = conduit_data.flow_max
  in_offset = conduit_data.offset_upstream
  out_offset = conduit_data.offset_downstream

  for xsection_name, xsection_data in inp["CONDUITS"].items():
    shape = xsection_data.shape
    geom1 = xsection_data.height
    geom2 = xsection_data.parameter_2
    geom3 = xsection_data.parameter_3
    geom4 = xsection_data.parameter_4

    # Get all possible sets of output sizes based on which diameters satisfy the parameters
    with open(available_sizes) as file:
      for line in file:
        size = float(line.strip())

        for n in trials():

          velocity = peak_flow / (math.pi()*size^2*0.25)

          if velocity >= vel_min and velocity <= vel_max:
            geom1 = size
          # Check GDocs after
          # Consider water flow in the data frame of Conduits

Optimization Test 1

In [ ]:
# Get initial peak flow rates (using your peak_flow_rates function or similar)
initial_peak_flows = peak_flow(rpt)  # Replace with your actual function call

# Define velocity constraints
vel_min = 0.8
vel_max = 5

# Function to find suitable sizes for a given conduit and peak flow
def find_suitable_sizes(conduit_name, peak_flow):
    suitable_sizes = []
    with open("Available Sizes.txt") as file:  # Assuming your available sizes are in this file
        for line in file:
            size = float(line.strip())
            velocity = peak_flow / (math.pi * size**2 * 0.25)
            if vel_min <= velocity <= vel_max:
                suitable_sizes.append(size)
    return suitable_sizes

# Optimization loop
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]

    # Optimize starting pipes simultaneously
    starting_pipes = [conduits_in_path[0]]  # Get the first conduit in the path

    suitable_sizes_per_pipe = {pipe: find_suitable_sizes(pipe, initial_peak_flows[pipe]) for pipe in starting_pipes}
    all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in starting_pipes]))

    for size_combination in all_size_combinations:
        modified_inp = inp.copy()

        # Modify pipe sizes for all starting pipes
        for i, pipe in enumerate(starting_pipes):
            modified_inp["CONDUITS"][pipe].height = size_combination[i]

        # Write modified input file and run simulation
        modified_inp.write_file("modified_input.inp")
        with Simulation("modified_input.inp") as sim:
            for step in sim:
                pass

        # Get updated peak flow rates for next conduits in the series
        # ... (use rpt object to get peak flow rates for downstream conduits) ...

        # Iterate downstream and optimize remaining conduits
        # ... (use similar logic as for starting pipes, but with updated peak flow rates) ...

Optimization Test 2

In [ ]:
# suitable_sizes = find_suitable_sizes(conduit_name, available_sizes, peak_flow)

# Optimization loop
# Get all starting pipes from all flow paths
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]
    all_starting_pipes.append(conduits_in_path[0])

# Run initial simulation to get initial peak flows
initial_peak_flows = peak_flow("Inlet_Drains_Model1.inp")

# Get suitable sizes for all starting pipes
suitable_sizes_per_pipe = {
    pipe: find_suitable_sizes(pipe, initial_peak_flows[pipe])
    for pipe in all_starting_pipes
}

# Generate all size combinations for all starting pipes
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))

# Optimization for all starting pipes simultaneously
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        modified_inp["CONDUITS"][pipe].height = size_combination[i]

    # Save modified inp to a separate file
    file_name = f"modified_input_{'_'.join(map(str, size_combination))}.inp"
    modified_inp.write_file(file_name)

    # Run simulation for the saved inp file
    updated_peak_flows = peak_flow(file_name)

    # ... (use updated_peak_flows to get peak flow rates for downstream conduits) ...

    # Iterate downstream and optimize remaining conduits
    # ... (use similar logic as for starting pipes, but with updated peak flow rates) ...


Optimization Test 3

In [ ]:
# Optimization loop
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]

    # --- Start with initial simulation ---
    current_peak_flows = peak_flow("Inlet_Drains_Model1.inp")  # Assuming initial input file name
    modified_inp = copy.deepcopy(inp)

    # --- Iterate through conduits in the path ---
    for i, conduit_name in enumerate(conduits_in_path):
        suitable_sizes = find_suitable_sizes(conduit_name, current_peak_flows[conduit_name])

        # --- Try each suitable size for the current conduit ---
        for size in suitable_sizes:
            modified_inp["CONDUITS"][conduit_name].height = size

            # Save modified inp to a separate file
            file_name = f"modified_input_{conduit_name}_{size}.inp"
            modified_inp.write_file(file_name)

            # Run simulation for the saved inp file
            current_peak_flows = peak_flow(file_name)  # Update peak flows for next conduit

            # ... (You can add logic here to compare results or store data) ...

        # --- After trying all sizes for current conduit, move to the next ---

    # --- After processing all conduits in the path, move to the next flow path ---